In [2]:
import os

import pandas as pd

In [3]:
# general configs
pd.set_option("display.max_columns", 500)

# DSS_SINI_AUTOS_ID - analysis

In [8]:
# setting configs and constants
# path to raw dataset
PATH = "../../../data/interim/1. col_red"
FILENAME = "DSS_SINI_AUTOS_ID-red_col.csv"
RAW_FILE = os.path.join(PATH, FILENAME)

if not(os.path.exists(RAW_FILE) and os.path.isfile(RAW_FILE)):
    raise Exception("File doesn't exists.")

## Loading database

In [16]:
usecols = ['CATASTROFICO', 'CERRADURA_BAUL', 'CERRADURA_DERECHA', 'CERRADURA_IZQUIERDA', 'CODIGO_BAJA', 'CODIGO_CARATULA', 'CODIGO_REAPERTURA', 'CODIGO_VEHICULO', 'COD_ACT_BENEF', 'COD_CAUSA_SINI', 'COD_POST_OCURRENCIA', 'COD_POST_POLIZA', 'COD_POST_TERC', 'COD_RAMO', 'COD_RIES', 'CONDICION_ROBO_EXP50', 'DANOS_MATERIALES', 'DESCRIPCION_TIPO', 'DESCRIPCION_VEHICULO', 'ESTADO_CIVIL', 'ESTADO_CIVIL_TERC', 'ESTAD_VEH_ASEG', 'FALTANTE', 'FECHA_FORMAL', 'FECHA_NAC_ASEG', 'FECHA_NAC_TERC', 'FECHA_SINI', 'FEC_DENU_SINI', 'FIN_INVESTIGACION', 'MCA_COASEG', 'MCA_JUICIO', 'MCA_VIP', 'METRO', 'NUM_SECU_EXPED', 'NUM_SECU_POL', 'OCUPACION_ASEG', 'SEXO', 'SEXO_TERC', 'TELEFONO_TERC', 'TIPO', 'TIPO_EXPED', 'TIPO_LESION', 'TIPO_LESION_MAXIMA', 'TIPO_SINIESTRO', 'USO', ]
df = pd.read_csv(RAW_FILE, usecols=usecols)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516974 entries, 0 to 7516973
Data columns (total 45 columns):
CATASTROFICO            object
CERRADURA_BAUL          object
CERRADURA_DERECHA       object
CERRADURA_IZQUIERDA     object
CODIGO_BAJA             float64
CODIGO_CARATULA         float64
CODIGO_REAPERTURA       float64
CODIGO_VEHICULO         float64
COD_ACT_BENEF           float64
COD_CAUSA_SINI          float64
COD_POST_OCURRENCIA     float64
COD_POST_POLIZA         float64
COD_POST_TERC           float64
COD_RAMO                int64
COD_RIES                float64
CONDICION_ROBO_EXP50    float64
DANOS_MATERIALES        object
DESCRIPCION_TIPO        object
DESCRIPCION_VEHICULO    object
ESTADO_CIVIL            object
ESTADO_CIVIL_TERC       object
ESTAD_VEH_ASEG          float64
FALTANTE                object
FECHA_FORMAL            object
FECHA_NAC_ASEG          object
FECHA_NAC_TERC          object
FECHA_SINI              object
FEC_DENU_SINI           object
FIN_INVE

In [8]:
df["NUM_SECU_EXPED"].nunique()

1144995

## formatting columns

In [40]:
to_date = ['FECHA_SINI', 'FEC_DENU_SINI', 'FECHA_NAC_ASEG', 'FECHA_NAC_TERC']
df[to_date].tail(10)

,FECHA_SINI,FEC_DENU_SINI,FECHA_NAC_ASEG,FECHA_NAC_TERC,FECHA_FORMAL
7516964,2019-04-13,2019-04-13,1969-12-20,NaT,NaN
7516965,2019-04-13,2019-04-13,1969-12-20,NaT,NaN
7516966,2019-04-13,2019-04-13,1969-12-20,NaT,NaN
7516967,2019-04-13,2019-04-13,1969-12-20,1976-12-23,NaN
7516968,2019-05-23,2019-05-24,1981-07-08,1999-01-01,NaN
7516969,2019-05-23,2019-05-24,1981-07-08,NaT,NaN
7516970,2019-05-23,2019-05-24,1981-07-08,NaT,NaN
7516971,2019-05-23,2019-05-24,1981-07-08,1999-01-01,NaN
7516972,2019-05-24,2019-05-27,1950-01-23,NaT,NaN
7516973,2019-05-24,2019-05-27,1950-01-23,NaT,NaN


In [18]:
%%time 
date_format = "%Y/%m/%d"
for col in to_date:
    df[col] = pd.to_datetime(df[col], format=date_format, errors="coerce")

CPU times: user 19.4 s, sys: 2.46 s, total: 21.9 s
Wall time: 19.4 s


In [42]:
date_format = "%d/%m/%Y"
col = "FECHA_FORMAL"
df[col] = pd.to_datetime(df[col], format=date_format, errors="coerce")

In [47]:
len(df[df['FECHA_SINI'] == df['FECHA_FORMAL']]), len(df[df['FEC_DENU_SINI'] == df['FECHA_FORMAL']]), len(df)

(56844, 176540, 7516974)

In [63]:
df["MCA_COASEG"] = df["MCA_COASEG"] == "SI"

In [68]:
df["SEXO_TERC"].value_counts(dropna=False) / len(df)

M      0.601065
NaN    0.317672
F      0.081263
Name: SEXO_TERC, dtype: float64

In [ ]:
#to_cat = ["MCA_VIP", "METRO", "CERRADURA_BAUL", "CERRADURA_DERECHA", "CERRADURA_IZQUIERDA", "SEXO", "SEXO_TERC"]
#df[to_cat] = pd.Categorical(df[to_cat]).codes+1

## creating new columns

In [52]:
df["dist_fformal_fsini"] = (df["FECHA_FORMAL"] - df["FECHA_SINI"]).dt.days
df["dist_fformal_fdenu"] = (df["FECHA_FORMAL"] - df["FEC_DENU_SINI"]).dt.days

In [74]:
# adding list to vars to drop after finishing the transformation
to_drop = ['FEC_DENU_SINI', 'FECHA_NAC_ASEG', 'FECHA_NAC_TERC', "FECHA_FORMAL", "FIN_INVESTIGACION"]

In [28]:
df["dias_entre_denu_y_sini"] = (df["FEC_DENU_SINI"] - df["FECHA_SINI"]).dt.days
df["dias_entre_denu_y_sini"].value_counts()[:10]

0    2242760
1    1902926
2     917552
3     618870
4     358375
5     237343
6     149561
7     111292
8      77157
9      59086
Name: dias_entre_denu_y_sini, dtype: int64

In [34]:
df["edad_aseg"] = df["FECHA_SINI"].dt.year - df['FECHA_NAC_ASEG'].dt.year
df["edad_terc"] = df["FECHA_SINI"].dt.year - df['FECHA_NAC_TERC'].dt.year
df['FECHA_NAC_ASEG'].head(20)

0    1948-07-13
1    1948-07-13
2    1992-06-14
3    1976-01-17
4    1987-07-28
5    1939-12-04
6    1955-01-12
7    1979-06-14
8    1974-01-15
9    1900-01-01
10   1964-09-17
11   1986-12-02
12   1981-10-05
13   1956-12-08
14   1900-01-01
15   1971-08-29
16   1969-01-09
17   1969-01-09
18   1969-01-09
19   1992-12-19
Name: FECHA_NAC_ASEG, dtype: datetime64[ns]

In [59]:
df["existe_FECHA_FORMAL"] = df["FECHA_FORMAL"].notna()

In [75]:
df.drop(columns=to_drop, inplace=True)

# saving into feather

In [77]:
# convert format to save into feather
df["TIPO_EXPED"] = df["TIPO_EXPED"].astype(str)

In [78]:
print(df["TIPO_EXPED"].unique())

['3' '10' '50' '2' '1' '60' '20' '8' '70' '003' '010' '060' '002' '001'
 '050' '020' 'G01' '008' 'G97' 'G98' '070' 'G99']


In [81]:
file_save = FILENAME.split(".")[0]
file_save

'DSS_SINI_AUTOS_ID-red_col'

In [79]:
file_save = FILENAME.split(".")[0]
to_save = f"../../../data/interim/4. transformed/{file_save}-newcols.feather"
df.to_feather(to_save)

# Checking data quality# Loading dataset

In [10]:
file_save = FILENAME.split(".")[0]
file = f"../../../data/interim/4. transformed/{file_save}-newcols.feather"

In [13]:
# path to tmp saved file
if not(os.path.exists(file) and os.path.isfile(file)):
    raise Exception("File doesn't exists.")

In [12]:
df = pd.read_feather(file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516974 entries, 0 to 7516973
Data columns (total 44 columns):
CATASTROFICO              object
CERRADURA_BAUL            object
CERRADURA_DERECHA         object
CERRADURA_IZQUIERDA       object
CODIGO_BAJA               float64
CODIGO_CARATULA           float64
CODIGO_REAPERTURA         float64
CODIGO_VEHICULO           float64
COD_ACT_BENEF             float64
COD_CAUSA_SINI            float64
COD_POST_OCURRENCIA       float64
COD_POST_POLIZA           float64
COD_POST_TERC             float64
COD_RAMO                  int64
COD_RIES                  float64
CONDICION_ROBO_EXP50      float64
DANOS_MATERIALES          object
DESCRIPCION_TIPO          object
DESCRIPCION_VEHICULO      object
ESTADO_CIVIL              object
ESTADO_CIVIL_TERC         object
ESTAD_VEH_ASEG            float64
FALTANTE                  object
FECHA_SINI                datetime64[ns]
MCA_COASEG                bool
MCA_JUICIO                object
MCA_VIP     

## removing duplicated rows

In [15]:
# checking duplicated rows
df.duplicated().sum()

4784543

In [17]:
df.drop_duplicates(inplace=True)
# checking duplicated rows
df.duplicated().sum()

0

### Saving checkpoint

In [31]:
df.reset_index(drop=True).to_feather(f"../../../data/interim/4. transformed/{file_save}-newcols-dropDuplic.feather")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2732431 entries, 0 to 7516972
Data columns (total 44 columns):
CATASTROFICO              object
CERRADURA_BAUL            object
CERRADURA_DERECHA         object
CERRADURA_IZQUIERDA       object
CODIGO_BAJA               float64
CODIGO_CARATULA           float64
CODIGO_REAPERTURA         float64
CODIGO_VEHICULO           float64
COD_ACT_BENEF             float64
COD_CAUSA_SINI            float64
COD_POST_OCURRENCIA       float64
COD_POST_POLIZA           float64
COD_POST_TERC             float64
COD_RAMO                  int64
COD_RIES                  float64
CONDICION_ROBO_EXP50      float64
DANOS_MATERIALES          object
DESCRIPCION_TIPO          object
DESCRIPCION_VEHICULO      object
ESTADO_CIVIL              object
ESTADO_CIVIL_TERC         object
ESTAD_VEH_ASEG            float64
FALTANTE                  object
FECHA_SINI                datetime64[ns]
MCA_COASEG                bool
MCA_JUICIO                object
MCA_VIP     

In [23]:
round(df.isna().sum() / len(df) * 100, 2)

CATASTROFICO               0.00
CERRADURA_BAUL            59.10
CERRADURA_DERECHA         59.10
CERRADURA_IZQUIERDA       59.10
CODIGO_BAJA               82.45
CODIGO_CARATULA           94.55
CODIGO_REAPERTURA         97.77
CODIGO_VEHICULO            0.01
COD_ACT_BENEF             69.65
COD_CAUSA_SINI             0.00
COD_POST_OCURRENCIA        0.00
COD_POST_POLIZA            0.00
COD_POST_TERC             78.29
COD_RAMO                   0.00
COD_RIES                   0.00
CONDICION_ROBO_EXP50      98.46
DANOS_MATERIALES           0.00
DESCRIPCION_TIPO           0.00
DESCRIPCION_VEHICULO       0.01
ESTADO_CIVIL               1.37
ESTADO_CIVIL_TERC         79.28
ESTAD_VEH_ASEG            15.51
FALTANTE                   0.00
FECHA_SINI                 0.00
MCA_COASEG                 0.00
MCA_JUICIO                94.55
MCA_VIP                    0.00
METRO                      0.00
NUM_SECU_EXPED             0.00
NUM_SECU_POL               0.00
OCUPACION_ASEG             0.08
SEXO    

In [19]:
# checking ID is unique
df["NUM_SECU_EXPED"].nunique() / len(df)

0.419038943709832

In [20]:
# checking ID is unique
df["NUM_SECU_POL"].nunique() / len(df)

0.26860403794276966

In [29]:
for k, v in df["NUM_SECU_EXPED"].value_counts().head().items():
    print(k, v)

1755223859999.0 22
1647862209999.0 21
1735813319999.0 20
1703856299999.0 18
1686966719999.0 17


In [30]:
df[df["NUM_SECU_EXPED"] == 1755223859999.0]

,CATASTROFICO,CERRADURA_BAUL,CERRADURA_DERECHA,CERRADURA_IZQUIERDA,CODIGO_BAJA,CODIGO_CARATULA,CODIGO_REAPERTURA,CODIGO_VEHICULO,COD_ACT_BENEF,COD_CAUSA_SINI,COD_POST_OCURRENCIA,COD_POST_POLIZA,COD_POST_TERC,COD_RAMO,COD_RIES,CONDICION_ROBO_EXP50,DANOS_MATERIALES,DESCRIPCION_TIPO,DESCRIPCION_VEHICULO,ESTADO_CIVIL,ESTADO_CIVIL_TERC,ESTAD_VEH_ASEG,FALTANTE,FECHA_SINI,MCA_COASEG,MCA_JUICIO,MCA_VIP,METRO,NUM_SECU_EXPED,NUM_SECU_POL,OCUPACION_ASEG,SEXO,SEXO_TERC,TELEFONO_TERC,TIPO,TIPO_EXPED,TIPO_LESION,TIPO_LESION_MAXIMA,TIPO_SINIESTRO,USO,dias_entre_denu_y_sini,dist_fformal_fsini,dist_fformal_fdenu,existe_FECHA_FORMAL
7142137,NO,None,None,None,NaN,25.0,NaN,36763.0,NaN,501.0,1022000.0,1665001.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,S,2017-10-09,False,M,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142143,NO,None,None,None,NaN,25.0,NaN,36763.0,7.0,501.0,1022000.0,1884018.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,N,2017-10-09,False,J,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142146,NO,None,None,None,NaN,25.0,NaN,36763.0,7.0,501.0,1022000.0,1427000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,S,2017-10-09,False,M,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142147,NO,None,None,None,NaN,25.0,NaN,36763.0,8.0,501.0,1022000.0,1033000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,S,2017-10-09,False,M,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142148,NO,None,None,None,NaN,25.0,NaN,36763.0,7.0,501.0,1022000.0,1882000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,S,2017-10-09,False,J,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142150,NO,None,None,None,NaN,25.0,NaN,36763.0,7.0,501.0,1022000.0,1426000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,N,2017-10-09,False,J,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142151,NO,None,None,None,NaN,25.0,NaN,36763.0,15.0,501.0,1022000.0,1426000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,N,2017-10-09,False,J,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142157,NO,None,None,None,NaN,25.0,NaN,36763.0,NaN,501.0,1022000.0,1876000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,N,2017-10-09,False,J,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142163,NO,None,None,None,NaN,25.0,NaN,36763.0,NaN,501.0,1022000.0,1426000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,N,2017-10-09,False,J,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True
7142166,NO,None,None,None,NaN,25.0,NaN,36763.0,NaN,501.0,1022000.0,1425000.0,1407000.0,8,1.0,NaN,"0,00","PICK-UPS DE CLASE ""A",RENAULT KANGOO 2 1.6 EX. 1 PLC,CA,SO,5.0,S,2017-10-09,False,C,S,S,1.755224e+12,1717801859999,999.0,M,M,4644-1601,8.0,2,L,L,B,16.0,1,46.0,45.0,True


In [37]:
df.drop_duplicates(subset=["NUM_SECU_EXPED"], keep="last", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144995 entries, 85 to 7516972
Data columns (total 44 columns):
CATASTROFICO              1144989 non-null object
CERRADURA_BAUL            504402 non-null object
CERRADURA_DERECHA         504402 non-null object
CERRADURA_IZQUIERDA       504402 non-null object
CODIGO_BAJA               1562 non-null float64
CODIGO_CARATULA           30240 non-null float64
CODIGO_REAPERTURA         58512 non-null float64
CODIGO_VEHICULO           1144905 non-null float64
COD_ACT_BENEF             463893 non-null float64
COD_CAUSA_SINI            1144995 non-null float64
COD_POST_OCURRENCIA       1144991 non-null float64
COD_POST_POLIZA           1144971 non-null float64
COD_POST_TERC             194056 non-null float64
COD_RAMO                  1144995 non-null int64
COD_RIES                  1144995 non-null float64
CONDICION_ROBO_EXP50      13085 non-null float64
DANOS_MATERIALES          1144989 non-null object
DESCRIPCION_TIPO          1144993 non-nu

In [38]:
# checking ID is unique
df["NUM_SECU_EXPED"].nunique() / len(df)

1.0

In [39]:
# checking ID is unique
df["NUM_SECU_POL"].nunique() / len(df)

0.6410001790400831

In [41]:
for k, v in df["NUM_SECU_EXPED"].value_counts().head().items():
    print(k, v)

1714103179999.0 1
1859805559999.0 1
1821712879999.0 1
1671587409999.0 1
1821720239999.0 1


In [40]:
df[df["NUM_SECU_EXPED"] == 1743575289999.0]

,CATASTROFICO,CERRADURA_BAUL,CERRADURA_DERECHA,CERRADURA_IZQUIERDA,CODIGO_BAJA,CODIGO_CARATULA,CODIGO_REAPERTURA,CODIGO_VEHICULO,COD_ACT_BENEF,COD_CAUSA_SINI,COD_POST_OCURRENCIA,COD_POST_POLIZA,COD_POST_TERC,COD_RAMO,COD_RIES,CONDICION_ROBO_EXP50,DANOS_MATERIALES,DESCRIPCION_TIPO,DESCRIPCION_VEHICULO,ESTADO_CIVIL,ESTADO_CIVIL_TERC,ESTAD_VEH_ASEG,FALTANTE,FECHA_SINI,MCA_COASEG,MCA_JUICIO,MCA_VIP,METRO,NUM_SECU_EXPED,NUM_SECU_POL,OCUPACION_ASEG,SEXO,SEXO_TERC,TELEFONO_TERC,TIPO,TIPO_EXPED,TIPO_LESION,TIPO_LESION_MAXIMA,TIPO_SINIESTRO,USO,dias_entre_denu_y_sini,dist_fformal_fsini,dist_fformal_fdenu,existe_FECHA_FORMAL
5405136,NO,None,None,None,NaN,NaN,NaN,18129.0,15.0,501.0,8336009.0,8336009.0,8336009.0,1,1.0,NaN,"18171,00","AUTOS, CAMIONE.RURAL",TOYOTA ETIOS 1.5 4P XS 6MT,CA,None,5.0,N,2017-10-30,False,None,S,N,1.743575e+12,1722359399999,5.0,F,M,None,3.0,003,None,None,N,1.0,0,1.0,1.0,True


In [42]:
df.reset_index(drop=True).to_feather(f"../../../data/interim/4. transformed/{file_save}-newcols-dropDuplic-NUM_SECU_EXPED_UNIC.feather")